# BIP0322 Verification

Initially this is just attempting to verify the test vectors provided in BIP0322

### Simple Signature


From the BIP-0322:

`A simple signature consists of a witness stack, consensus encoded as a vector of vectors of bytes, and base64-encoded. Validators should construct to_spend and to_sign as defined below, with default values for all fields except that`

### Test Vector Sig

This signature is the test vector taken from [bip0322](https://github.com/bitcoin/bips/blob/master/bip-0322.mediawiki#message-signing).

**Currently unable to get this to verify.**

In [1]:
simple_signature = 'AkcwRAIgG3PASL/vRTgAqogWT6S8rUOQXNnfRzX6JncmbFlHc1ACIGQdsW+rnVmsQzyAYRQisHKFMigDmKiL7LUw4x17Fw5tASECx/EgAxlkQpQ9hYjgGu6EBCPMVPwVIVJqO4XCsMvViHI='

### Produced Sig

This signature was produced in the [BIP0322_signing](http://localhost:8888/notebooks/BIP0322_signing.ipynb) notebook. Currently it is this signature that verifies against the message and signing address.

**Note: It seems likely I am just replicating the same mistake across both notebooks?**

In [46]:
produced_sig = 'AkgwRQIhAOzyynlqt93lOKJr+wmmxIens//zPzl9tqIOua93wO6MAiBi5n5EyAcPScOjf1lAqIUIQtr3zKNeavYabHyR8eGhowEhAsfxIAMZZEKUPYWI4BruhAQjzFT8FSFSajuFwrDL1Yhy'

### Signing Address

In [3]:
# Test vector taken from the bip
address = 'bc1q9vza2e8x573nczrlzms0wvx3gsqjx7vavgkx0l'

### Message Signed

In [4]:
from buidl.hash import tagged_hash
from buidl.helper import hash256
message = b"Hello World"
tag = b"BIP0322-signed-message"

tagged_hash = tagged_hash(tag,message)

test_vector = 'f0eb03b1a75ac6d9847f55c624a99169b5dccba2a31f5b23bea77ba270de0a7a'
assert(test_vector == tagged_hash.hex())

# Bitcoin Core Implementation Results

### The Test Case

```c
BOOST_CHECK_EQUAL(
    MessageVerify(
        "bc1q9vza2e8x573nczrlzms0wvx3gsqjx7vavgkx0l",
        "AkcwRAIgZRfIY3p7/DoVTty6YZbWS71bc5Vct9p9Fia83eRmw2QCICK/ENGfwLtptFluMGs2KsqoNSk89pO7F29zJLUx9a/sASECx/EgAxlkQpQ9hYjgGu6EBCPMVPwVIVJqO4XCsMvViHI=",
        "Hello World"),
    MessageVerificationResult::OK);
```

### Command Line Output

Output generated by adding the following cout print statements to MessageVerifyBIP322. See https://github.com/bitcoin/bitcoin/blob/843ce398a1d4661b9d93160635b46cc0a70d6153/src/util/message.cpp#L48 for original

```c
MessageVerificationResult MessageVerifyBIP322(
    CTxDestination& destination,
    std::vector<unsigned char>& signature,
    const std::string& message,
    MessageVerificationResult legacyError)
{
    auto txs = BIP322Txs::Create(destination, message, legacyError, &signature);
    if (!txs) return legacyError;

    const CTransaction& to_sign = txs->m_to_sign;
    const CTransaction& to_spend = txs->m_to_spend;

    std::cout << "This is the to_spend tx : \n" << to_spend.ToString();
    std::cout << "\nThis is the to_spend tx hash : \n" << to_spend.GetHash().ToString();

    std::cout << "\n\n This is the to_spend tx : \n" << to_sign.ToString();

    std::cout << "\nThis is the to_sign tx hash : \n" << to_sign.GetHash().ToString();

    const CScript scriptSig = to_sign.vin[0].scriptSig;
    const CScriptWitness& witness = to_sign.vin[0].scriptWitness;

    PrecomputedTransactionData txdata;
    txdata.Init(to_sign, {to_spend.vout[0]});
    TransactionSignatureChecker sigcheck(&to_sign, /* nInIn= */ 0, /* amountIn= */ to_spend.vout[0].nValue, txdata, MissingDataBehavior::ASSERT_FAIL);
    sigcheck.RequireSighashAll();

    if (!VerifyScript(scriptSig, to_spend.vout[0].scriptPubKey, &witness, BIP322_REQUIRED_FLAGS, sigcheck)) {
        return MessageVerificationResult::ERR_INVALID;
    }

    // inconclusive checks

    if (to_sign.nVersion != 0 && to_sign.nVersion != 2) {
        return MessageVerificationResult::INCONCLUSIVE;
    }

    if (!VerifyScript(scriptSig, to_spend.vout[0].scriptPubKey, &witness, BIP322_INCONCLUSIVE_FLAGS, sigcheck)) {
        return MessageVerificationResult::INCONCLUSIVE;
    }

    // timelock check
    if (to_sign.nLockTime > 0 || to_sign.vin[0].nSequence > 0) {
        return MessageVerificationResult::OK_TIMELOCKED;
    }



    return MessageVerificationResult::OK;
}
```

### The Resulting Output

```bash
This is the to_spend tx : 
CTransaction(hash=b79d196740, ver=0, vin.size=1, vout.size=1, nLockTime=0)
    CTxIn(COutPoint(0000000000, 4294967295), coinbase 0020f0eb03b1a75ac6d9847f55c624a99169b5dccba2a31f5b23bea77ba270de0a7a, nSequence=0)
    CScriptWitness()
    CTxOut(nValue=0.00000000, scriptPubKey=00142b05d564e6a7a33c087f16e0f7)

This is the to_spend tx hash : 
b79d196740ad5217771c1098fc4a4b51e0535c32236c71f1ea4d61a2d603352b

 This is the to_spend tx : 
CTransaction(hash=88737ae86f, ver=0, vin.size=1, vout.size=1, nLockTime=0)
    CTxIn(COutPoint(b79d196740, 0), scriptSig=, nSequence=0)
    CScriptWitness(304402206517c8637a7bfc3a154edcba6196d64bbd5b73955cb7da7d1626bcdde466c364022022bf10d19fc0bb69b4596e306b362acaa835293cf693bb176f7324b531f5afec01, 02c7f12003196442943d8588e01aee840423cc54fc1521526a3b85c2b0cbd58872)
    CTxOut(nValue=0.00000000, scriptPubKey=6a)

This is the to_sign tx hash : 
88737ae86f2077145f93cc4b153ae9a1cb8d56afa511988c149c5c8c9d93bddf

info: check MessageVerify( "bc1q9vza2e8x573nczrlzms0wvx3gsqjx7vavgkx0l", "AkcwRAIgZRfIY3p7/DoVTty6YZbWS71bc5Vct9p9Fia83eRmw2QCICK/ENGfwLtptFluMGs2KsqoNSk89pO7F29zJLUx9a/sASECx/EgAxlkQpQ9hYjgGu6EBCPMVPwVIVJqO4XCsMvViHI=", "Hello World") == MessageVerificationResult::OK has passed
```

### Bitcoin core generated tx hashes

**Currently neither of these hashes match the impl below.**

In [7]:
btc_to_spend_tx_hash = "b79d196740ad5217771c1098fc4a4b51e0535c32236c71f1ea4d61a2d603352b"
btx_to_sign_tx_hash = "88737ae86f2077145f93cc4b153ae9a1cb8d56afa511988c149c5c8c9d93bddf"

## Identified Differences compared to Below Implementation

### The test vector

Most strikingly the test vector that Bitcoin core uses to get a `MessageVerificationResult::OK` on the message "Hello World" is **NOT** the same as the BIP!

See [line 2596 of util_tests.cpp](https://github.com/bitcoin/bitcoin/blob/843ce398a1d4661b9d93160635b46cc0a70d6153/src/test/util_tests.cpp#L2596):

```c
    BOOST_CHECK_EQUAL(
        MessageVerify(
            "bc1q9vza2e8x573nczrlzms0wvx3gsqjx7vavgkx0l",
            "AkcwRAIgZRfIY3p7/DoVTty6YZbWS71bc5Vct9p9Fia83eRmw2QCICK/ENGfwLtptFluMGs2KsqoNSk89pO7F29zJLUx9a/sASECx/EgAxlkQpQ9hYjgGu6EBCPMVPwVIVJqO4XCsMvViHI=",
            "Hello World"),
        MessageVerificationResult::OK);
```


In [8]:
btc_core_test_vector_sig = "AkcwRAIgZRfIY3p7/DoVTty6YZbWS71bc5Vct9p9Fia83eRmw2QCICK/ENGfwLtptFluMGs2KsqoNSk89pO7F29zJLUx9a/sASECx/EgAxlkQpQ9hYjgGu6EBCPMVPwVIVJqO4XCsMvViHI="

assert(simple_signature == btc_core_test_vector_sig)

AssertionError: 

### Although neither is it the same as the signature I produced in the signing notebook

In [11]:
assert(produced_sig == btc_core_test_vector_sig)

AssertionError: 

## to_spend transaction

Bitcoin Core output when printing the to_spend.ToString() result

################################################################################

```bash
CTransaction(hash=b79d196740, ver=0, vin.size=1, vout.size=1, nLockTime=0)
    CTxIn(COutPoint(0000000000, 4294967295), coinbase 0020f0eb03b1a75ac6d9847f55c624a99169b5dccba2a31f5b23bea77ba270de0a7a, nSequence=0)
    CScriptWitness()
    CTxOut(nValue=0.00000000, scriptPubKey=00142b05d564e6a7a33c087f16e0f7)
```
################################################################################

### Differences

**THERE DON'T APPEAR TO BE ANY**

- CTxIn appears as a coinbase in the print, but so is the tx I produced (See below)
- nValue in CTxOut is in decimal. 
    - Don't think this should matter

### to_sign transaction

Bitcoin Core output

##########################################################################

```bash
CTransaction(hash=88737ae86f, ver=0, vin.size=1, vout.size=1, nLockTime=0)
    CTxIn(COutPoint(b79d196740, 0), scriptSig=, nSequence=0)
    CScriptWitness(304402206517c8637a7bfc3a154edcba6196d64bbd5b73955cb7da7d1626bcdde466c364022022bf10d19fc0bb69b4596e306b362acaa835293cf693bb176f7324b531f5afec01, 02c7f12003196442943d8588e01aee840423cc54fc1521526a3b85c2b0cbd58872)
    CTxOut(nValue=0.00000000, scriptPubKey=6a)
```
##########################################################################

### Differences

- The witness is different from both the witness encoded as a signature and provided as a test vector and the witness I generated in this impl.
- Furthermore, when I decode and parse the Bitcoin Core signature I do not get the same Witness as the output above

# Verification Process

## 1. Basic Validation

### 1a. Compute virtual to_spend transaction
```
    nVersion = 0
    nLockTime = 0
    vin[0].prevout.hash = 0000...000
    vin[0].prevout.n = 0xFFFFFFFF
    vin[0].nSequence = 0
    vin[0].scriptSig = OP_0 PUSH32[ message_hash ]
    vin[0].scriptWitness = []
    vout[0].nValue = 0
    vout[0].scriptPubKey = message_challenge
```

In [12]:
from buidl.tx import Tx, TxIn, TxOut
from buidl.helper import little_endian_to_int
from buidl.script import Script,ScriptPubKey

In [16]:
# Not a valid Tx hash. Will never be spendable on any BTC network.
prevout_hash = bytes.fromhex('0000000000000000000000000000000000000000000000000000000000000000')
# prevout.n
prevout_index = little_endian_to_int(bytes.fromhex('FFFFFFFF'))
sequence = 0

# Byte array of message hash
message_hash = tagged_hash

# Spotted the error
# OP_PUSH32 gets automatically added to the stack when add 32 bytes of data
# commands = [0, 32, message_hash]
commands = [0, message_hash]
scriptSig = Script(commands)

# This is equivalent to the one provided
print(scriptSig.raw_serialize().hex())

assert("0020f0eb03b1a75ac6d9847f55c624a99169b5dccba2a31f5b23bea77ba270de0a7a" == scriptSig.raw_serialize().hex())

# Create Tx Input
tx_in = TxIn(prevout_hash,prevout_index,scriptSig,sequence)

0020f0eb03b1a75ac6d9847f55c624a99169b5dccba2a31f5b23bea77ba270de0a7a


### It is weird (wrong?) that the Script contained in Bitcoin Core does not include the PUSH32 opcode when it is clearly stated as required in the BIP

Note: This has no affect on the signature evaluation. Other than you MUST replicate the to_spend scriptSig accurately. **That scriptSig is never interpretted by the Bitcoin stack**

Obviously having a different scriptSig, changes the hashed tx and hence impacts any BIP322 signature results

In [25]:
btc_core_to_spend_script_sig = Script.parse_hex('0020f0eb03b1a75ac6d9847f55c624a99169b5dccba2a31f5b23bea77ba270de0a7a')
btc_core_to_spend_script_sig

OP_0 f0eb03b1a75ac6d9847f55c624a99169b5dccba2a31f5b23bea77ba270de0a7a 

In [26]:
from buidl.script import address_to_script_pubkey

# Value of tx output
value = 0

# Convert address to ScriptPubKey
script_pubkey = address_to_script_pubkey(address)

tx_out = TxOut(value,script_pubkey)

# Note: This is the same as the Bitcoin core
print("Implementation scriptpubkey hex representation", script_pubkey.raw_serialize().hex())
print("BitcoinCore scriptpubkey hex representation", "00142b05d564e6a7a33c087f16e0f7")


Implementation scriptpubkey hex representation 00142b05d564e6a7a33c087f16e0f730d1440123799d
BitcoinCore scriptpubkey hex representation 00142b05d564e6a7a33c087f16e0f7


In [27]:
tx_out

0:OP_0 2b05d564e6a7a33c087f16e0f730d1440123799d 

In [28]:
# create transaction
version=0
tx_inputs = [tx_in]
tx_outputs = [tx_out]
locktime=0
network="mainnet"

# Could be false, but using a segwit address. I think this is the "Simple Signature" in BIP-0322
segwit=True

virtual_to_spend_tx = Tx(version,tx_inputs,tx_outputs,locktime,network,segwit)

### This is equivalent to the txid from the Bitcoin core impl

In [31]:
print(virtual_to_spend_tx.id())
assert(virtual_to_spend_tx.id() == btc_to_spend_tx_hash)

b79d196740ad5217771c1098fc4a4b51e0535c32236c71f1ea4d61a2d603352b


In [32]:
print(virtual_to_spend_tx.is_coinbase())

True


### 1b.Decode the signature as the transaction virtual to_sign

Note: the signature in simple format is just the witness stack

```python
    nVersion = 0 or (FULL format only) as appropriate (e.g. 2, for time locks)
    nLockTime = 0 or (FULL format only) as appropriate (for time locks)
    vin[0].prevout.hash = to_spend.txid
    vin[0].prevout.n = 0
    vin[0].nSequence = 0 or (FULL format only) as appropriate (for time locks)
    vin[0].scriptWitness = message_signature
    vout[0].nValue = 0
    vout[0].scriptPubKey = OP_RETURN
```

#### Decode signature into witness stack using test vector

In [84]:
from buidl.helper import base64_decode
from buidl.ecc import Signature
from buidl.witness import Witness

# Note: Using the test vector I am unable to get the code to verify
decoded_witness_test = base64_decode(simple_signature)


import io
stream = io.BytesIO(decoded_witness_test)
witness_test = Witness.parse(stream)
print(witness_test)

304402201b73c048bfef453800aa88164fa4bcad43905cd9df4735fa2677266c594773500220641db16fab9d59ac433c80611422b0728532280398a88becb530e31d7b170e6d01 02c7f12003196442943d8588e01aee840423cc54fc1521526a3b85c2b0cbd58872 


#### Decode signature into witness stack using produced signature

In [47]:
from buidl.helper import base64_decode
from buidl.ecc import Signature
from buidl.witness import Witness

# Using the signature produced in the other notebook does work however
decoded_witness_produced = base64_decode(produced_sig)

import io
stream = io.BytesIO(decoded_witness_produced)
witness_produced = Witness.parse(stream)
print(witness_produced)

3045022100ecf2ca796ab7dde538a26bfb09a6c487a7b3fff33f397db6a20eb9af77c0ee8c022062e67e44c8070f49c3a37f5940a8850842daf7cca35e6af61a6c7c91f1e1a1a301 02c7f12003196442943d8588e01aee840423cc54fc1521526a3b85c2b0cbd58872 


#### Decode signature into witness stack using the btc core test vector witness

In [36]:
from buidl.helper import base64_decode
from buidl.ecc import Signature
from buidl.witness import Witness

# Using the signature produced in the other notebook does work however
decoded_witness_btc = base64_decode(btc_core_test_vector_sig)

import io
stream = io.BytesIO(decoded_witness_btc)
witness_btc = Witness.parse(stream)
print(witness_btc)

304402206517c8637a7bfc3a154edcba6196d64bbd5b73955cb7da7d1626bcdde466c364022022bf10d19fc0bb69b4596e306b362acaa835293cf693bb176f7324b531f5afec01 02c7f12003196442943d8588e01aee840423cc54fc1521526a3b85c2b0cbd58872 


#### Create tx_in user test vector witness

In [35]:
prevout_hash = virtual_to_spend_tx.hash()
prevout_index = 0
sequence = 0


tx_in_test = TxIn(prevout_hash,prevout_index,sequence=sequence)

# set the witness
tx_in_test.witness = witness_test

tx_in_test.witness

NameError: name 'witness_test' is not defined

#### Create tx_in using notebook produced witness

In [48]:
prevout_hash = virtual_to_spend_tx.hash()
prevout_index = 0
sequence = 0

tx_in_produced = TxIn(prevout_hash,prevout_index,sequence=sequence)

# set the witness
tx_in_produced.witness = witness_produced

tx_in_produced.witness

3045022100ecf2ca796ab7dde538a26bfb09a6c487a7b3fff33f397db6a20eb9af77c0ee8c022062e67e44c8070f49c3a37f5940a8850842daf7cca35e6af61a6c7c91f1e1a1a301 02c7f12003196442943d8588e01aee840423cc54fc1521526a3b85c2b0cbd58872 

#### Create tx_in using notebook btc_test_vector witness

In [41]:
prevout_hash = virtual_to_spend_tx.hash()
prevout_index = 0
sequence = 0

tx_in_btc_core = TxIn(prevout_hash,prevout_index,sequence=sequence)

# set the witness
tx_in_btc_core.witness = witness_btc

tx_in_btc_core.witness

304402206517c8637a7bfc3a154edcba6196d64bbd5b73955cb7da7d1626bcdde466c364022022bf10d19fc0bb69b4596e306b362acaa835293cf693bb176f7324b531f5afec01 02c7f12003196442943d8588e01aee840423cc54fc1521526a3b85c2b0cbd58872 

#### Create tx_out (same for both test and produced)

In [49]:
value = 0
# OP Code 106 for OP_RETURN
commands = [106]
scriptPubKey = Script(commands)

tx_output = TxOut(value,scriptPubKey)
tx_output

0:OP_RETURN 

#### Create virtual_to_sign_tx for test vector

In [39]:
version = 0
locktime = 0
virtual_to_sign_tx_test_vector = Tx(version, [tx_in_test], [tx_output],locktime,segwit=False)

print(virtual_to_sign_tx_test_vector.id())

88737ae86f2077145f93cc4b153ae9a1cb8d56afa511988c149c5c8c9d93bddf


#### Create virtual_to_sign_tx for produced signature

In [50]:
version = 0
locktime = 0
virtual_to_sign_tx_produced = Tx(version, [tx_in_produced], [tx_output],locktime,segwit=False)
print(virtual_to_sign_tx_test_vector.id())

88737ae86f2077145f93cc4b153ae9a1cb8d56afa511988c149c5c8c9d93bddf


#### Create virtual_to_sign_tx for btc_core_test_vector signature

In [43]:
version = 0
locktime = 0
virtual_to_sign_tx_bitcoin_core = Tx(version, [tx_in_btc_core], [tx_output],locktime,segwit=False)
print(virtual_to_sign_tx_bitcoin_core.id())

88737ae86f2077145f93cc4b153ae9a1cb8d56afa511988c149c5c8c9d93bddf


### A whole bunch of validation that SHOULD happen ...

### Initially all I am trying to do is validate the signature on the input in the to_sign transaction


### The input for the virtual to_sign transation IS the output at index 0 from the virtual to_spend transaction

Normally the relevant information (The outputs script_pubkey and value) would be populated by querying the network for the transaction and fetching it from there. In this case we set those values on the input manually.

# Verification of Test Vector

In [42]:
virtual_to_sign_tx_test_vector.tx_ins[0]._script_pubkey = script_pubkey
virtual_to_sign_tx_test_vector.tx_ins[0]._value = 0

### Verify the input is a valid unlocking of the to_spend output

Not sure why this isn't working for the test vector of BIP-0322

In [44]:
virtual_to_sign_tx_test_vector.verify_input(0)

False

# Verification of Notebook Produced Signature

In [51]:
virtual_to_sign_tx_produced.tx_ins[0]._script_pubkey = script_pubkey
virtual_to_sign_tx_produced.tx_ins[0]._value = 0

### Verify the input is a valid unlocking of the to_spend output

Not sure why this isn't working for the test vector of BIP-0322

In [52]:
virtual_to_sign_tx_produced.verify_input(0)

True

# Verification of Notebook Bitcoin Core Signature

In [44]:
virtual_to_sign_tx_bitcoin_core.tx_ins[0]._script_pubkey = script_pubkey
virtual_to_sign_tx_bitcoin_core.tx_ins[0]._value = 0

### Verify the input is a valid unlocking of the to_spend output

Not sure why this isn't working for the test vector of BIP-0322

In [45]:
virtual_to_sign_tx_bitcoin_core.verify_input(0)

True